In [44]:
import numpy as np
import tensorflow as tf
from IPython.display import display

In [45]:
import os
filename = 'age_pred.h5'
LOAD_MODEL = os.path.isfile('age_pred.h5')

In [46]:
path = './datasets/20-50'
train_ds = tf.keras.utils.image_dataset_from_directory(
  path + '/' + 'train',
  image_size=(128, 128),
  validation_split=0.2,
  subset='training',
  seed=1
)
val_ds = tf.keras.utils.image_dataset_from_directory(
  path + '/' + 'train',
  image_size=(128, 128),
  validation_split=0.2,
  subset='validation',
  seed=1
)
test_ds = tf.keras.utils.image_dataset_from_directory(
  path + '/' + 'test', 
  image_size=(128, 128),
)

Found 33432 files belonging to 31 classes.
Using 26746 files for training.
Found 33432 files belonging to 31 classes.
Using 6686 files for validation.
Found 7008 files belonging to 31 classes.


In [47]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

In [48]:
if not LOAD_MODEL:
    model = tf.keras.Sequential(
        layers = [
            tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(128, 128, 3)),
            tf.keras.layers.AveragePooling2D(pool_size=(2,2)),

            tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
            tf.keras.layers.AveragePooling2D(pool_size=(2,2)),

            tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'),
            tf.keras.layers.AveragePooling2D(pool_size=(2,2)),

            tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu'),
            tf.keras.layers.AveragePooling2D(pool_size=(2,2)),

            tf.keras.layers.GlobalAveragePooling2D(),

            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),

            tf.keras.layers.Dense(1, activation='linear'),
        ]
    )

    model.compile(
        optimizer='adam',
        loss='mse'
    )

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=100,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=5,
                restore_best_weights=True
            )
        ]
    )

    model.save(filename)
else:
    model = tf.keras.models.load_model(filename)

In [49]:
rmse = np.sqrt(model.evaluate(test_ds, verbose=0))
print(f'Test RMSE: {rmse}')

2022-05-19 15:30:26.417147: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Test RMSE: 8.988262576034492


In [50]:
predictions = model.predict(test_ds)
labels = np.array([np.array([label]) for _, label in test_ds.unbatch().as_numpy_iterator()])

2022-05-19 15:30:28.985103: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [51]:
for i in range(len(predictions)):
  print(f'Predicted age: {predictions[i][0]+20}')
  print(f'Actual age: {labels[i][0]+20}')
  print()

Predicted age: 33.951820373535156
Actual age: 28

Predicted age: 34.53612041473389
Actual age: 25

Predicted age: 33.87744140625
Actual age: 30

Predicted age: 34.696998596191406
Actual age: 31

Predicted age: 34.12797927856445
Actual age: 31

Predicted age: 34.56070899963379
Actual age: 42

Predicted age: 35.01284599304199
Actual age: 41

Predicted age: 34.68823432922363
Actual age: 40

Predicted age: 33.839558601379395
Actual age: 21

Predicted age: 34.915489196777344
Actual age: 20

Predicted age: 35.21364212036133
Actual age: 48

Predicted age: 34.52715492248535
Actual age: 22

Predicted age: 34.1866397857666
Actual age: 40

Predicted age: 35.022422790527344
Actual age: 36

Predicted age: 34.65038871765137
Actual age: 41

Predicted age: 34.470970153808594
Actual age: 35

Predicted age: 34.803436279296875
Actual age: 41

Predicted age: 34.4272403717041
Actual age: 26

Predicted age: 35.2610445022583
Actual age: 44

Predicted age: 33.70196533203125
Actual age: 28

Predicted age: 35.2